# Hybrid Autoencoder + CNN — Lokal/Colab + Google Drive Sync + Keras Tuner (LR)

Notebook singkat, readable, dan siap jalan di lokal atau Colab.

## 1) Cek versi & GPU

In [ ]:
import sys
import platform
import tensorflow as tf

print("Python     :", sys.version)
print("Platform   :", platform.platform())
print("TensorFlow :", tf.__version__)
print("GPU        :", tf.config.list_physical_devices('GPU'))

## 2) Konfigurasi data & mode

In [ ]:
from pathlib import Path

# Pilih sumber data: LOCAL, COLAB_DRIVE, atau GDRIVE_SYNC
DATA_SOURCE = "GDRIVE_SYNC"

# Path untuk setiap sumber data
LOCAL_DATA_DIR = r"./dataset_babybrandedshop"
COLAB_DRIVE_DIR = "/content/drive/MyDrive/dataset_babybrandedshop"
GDRIVE_FOLDER_ID = "1dtwyooh7nl5hivsFBGOL3qG6q9HF7xYF"

# Buat folder cache untuk sinkronisasi Google Drive
SYNC_CACHE_DIR = Path("./gdrive_cache_dataset")
SYNC_CACHE_DIR.mkdir(exist_ok=True)

## 3) Utilitas sinkronisasi Google Drive (lokal)

In [ ]:
CREDENTIAL = "client_secret_376472766929-sa1l06mdun2qkoi3o20vtslr103mkspi.apps.googleusercontent.com.json"

def sync_gdrive_folder(folder_id: str, target_dir: Path):
    """Download seluruh folder dari Google Drive ke lokal"""
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    
    # Validasi folder ID
    assert folder_id, "GDRIVE_FOLDER_ID kosong."
    
    # Autentikasi Google Drive
    gauth = GoogleAuth()
    gauth.LoadCredentialsFile(CREDENTIAL)
    
    if gauth.credentials is None:
        gauth.LocalWebserverAuth()
        gauth.SaveCredentialsFile(CREDENTIAL)
    elif gauth.access_token_expired:
        gauth.Refresh()
        gauth.SaveCredentialsFile(CREDENTIAL)
    else:
        gauth.Authorize()
    
    drive = GoogleDrive(gauth)
    
    # Fungsi helper untuk membuat direktori
    def ensure_dir(p: Path):
        p.mkdir(parents=True, exist_ok=True)
    
    # Fungsi untuk list file/folder dalam folder tertentu
    def list_children(fid: str):
        query = f"'{fid}' in parents and trashed=false"
        return drive.ListFile({'q': query}).GetList()
    
    # Fungsi rekursif untuk download folder dan isinya
    def recursive_download(fid: str, dest: Path):
        ensure_dir(dest)
        for f in list_children(fid):
            # Jika folder, download rekursif
            if f['mimeType'] == 'application/vnd.google-apps.folder':
                recursive_download(f['id'], dest / f['title'])
            # Jika file, download langsung
            else:
                out = dest / f['title']
                if not out.exists():
                    f.GetContentFile(str(out))
    
    recursive_download(folder_id, target_dir)
    return target_dir

print("Fungsi sync_gdrive_folder() siap digunakan")

## 4) Tentukan DATA_DIR

In [ ]:
# Cek apakah sedang berjalan di Google Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Tentukan DATA_DIR berdasarkan sumber data
if DATA_SOURCE == "LOCAL":
    # Gunakan dataset lokal
    DATA_DIR = Path(LOCAL_DATA_DIR).resolve()
    
elif DATA_SOURCE == "COLAB_DRIVE":
    # Gunakan Google Drive di Colab
    assert IN_COLAB, "Mode ini hanya untuk Google Colab."
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
    DATA_DIR = Path(COLAB_DRIVE_DIR).resolve()
    
elif DATA_SOURCE == "GDRIVE_SYNC":
    # Sinkronisasi dari Google Drive ke lokal
    DATA_DIR = sync_gdrive_folder(GDRIVE_FOLDER_ID, Path("./gdrive_cache_dataset")).resolve()
    
else:
    raise ValueError("DATA_SOURCE tidak valid. Pilih: LOCAL, COLAB_DRIVE, atau GDRIVE_SYNC")

print("DATA_DIR:", DATA_DIR)
assert DATA_DIR.exists(), f"Folder {DATA_DIR} tidak ditemukan!"
print("Dataset aman!")

## 5) Dataset TF

In [ ]:
import tensorflow as tf

# Konfigurasi ukuran gambar dan batch
IMG_SIZE = (256, 256)
BATCH_SIZE = 32

# Load dataset training (80% data)
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Load dataset validasi (20% data)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Optimasi performa dengan prefetch dan cache
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE).cache()
val_ds = val_ds.prefetch(AUTOTUNE).cache()

# Deteksi kelas dari folder
classes = sorted([p.name for p in Path(DATA_DIR).iterdir() if p.is_dir()])
NUM_CLASSES = len(classes)

print(f"Jumlah kelas: {NUM_CLASSES}")
print(f"Nama kelas: {classes}")

## 6) Augmentasi & Normalisasi

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Layer augmentasi data (untuk variasi gambar saat training)
augment = keras.Sequential([
    layers.RandomFlip(mode="horizontal_and_vertical"),  # Flip horizontal & vertikal
    layers.RandomRotation(factor=0.0833, fill_mode="reflect"),  # Rotasi ±30°
    layers.RandomZoom(height_factor=(-0.10, 0.10), width_factor=(-0.10, 0.10)),  # Zoom ±10%
    layers.RandomContrast(factor=0.2),  # Ubah kontras
    layers.RandomBrightness(factor=0.1),  # Ubah kecerahan
], name="augment")

# Layer preprocessing (resize dan normalisasi)
preprocess = keras.Sequential([
    layers.Resizing(IMG_SIZE[0], IMG_SIZE[1]),  # Resize ke ukuran yang sama
    layers.Rescaling(1./255),  # Normalisasi pixel dari 0-255 ke 0-1
], name="preprocess")

print("Augmentasi dan preprocessing done")

## 7) Bangun Autoencoder (Encoder + Decoder)

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow import keras

def build_encoder(input_shape=(256, 256, 3), latent_dim=128):
    """
    Bangun Encoder: kompres gambar menjadi representasi fitur (latent vector)
    Input: gambar 256x256x3
    Output: vector 128 dimensi
    """
    inputs = keras.Input(shape=input_shape)
    x = preprocess(inputs)
    
    # Layer 1: 256x256x3 -> 128x128x32
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Layer 2: 128x128x32 -> 64x64x64
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Layer 3: 64x64x64 -> 32x32x128
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Layer 4: 32x32x128 -> 16x16x256
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Flatten dan kompres ke latent vector
    x = layers.Flatten()(x)
    latent = layers.Dense(latent_dim, activation='relu', name='latent')(x)
    
    return Model(inputs, latent, name='encoder')


def build_decoder(output_shape=(256, 256, 3), latent_dim=128):
    """
    Bangun Decoder: rekonstruksi gambar dari latent vector
    Input: vector 128 dimensi
    Output: gambar 256x256x3
    """
    H, W, C = output_shape
    start_h, start_w, start_c = 16, 16, 256  # Ukuran awal: 16x16x256
    
    latent_in = keras.Input(shape=(latent_dim,))
    x = layers.Dense(start_h * start_w * start_c, activation='relu')(latent_in)
    x = layers.Reshape((start_h, start_w, start_c))(x)
    
    # Layer 1: 16x16x256 -> 32x32x256
    x = layers.Conv2DTranspose(256, 3, strides=2, activation='relu', padding='same')(x)
    
    # Layer 2: 32x32x256 -> 64x64x128
    x = layers.Conv2DTranspose(128, 3, strides=2, activation='relu', padding='same')(x)
    
    # Layer 3: 64x64x128 -> 128x128x64
    x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
    
    # Layer 4: 128x128x64 -> 256x256x32
    x = layers.Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')(x)
    
    # Output layer: 256x256x32 -> 256x256x3
    outputs = layers.Conv2D(C, 3, activation='sigmoid', padding='same')(x)
    
    return Model(latent_in, outputs, name='decoder')


# Bangun encoder dan decoder
encoder = build_encoder(IMG_SIZE + (3,), latent_dim=128)
decoder = build_decoder(IMG_SIZE + (3,), latent_dim=128)

# Gabungkan encoder + decoder menjadi autoencoder
inp = keras.Input(shape=IMG_SIZE + (3,))
z = encoder(inp)  # Encode gambar
recon = decoder(z)  # Decode kembali
autoencoder = Model(inp, recon, name='autoencoder')

# Compile autoencoder
autoencoder.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='mse'  # Mean Squared Error untuk rekonstruksi
)

print("\nArsitektur Autoencoder:")
autoencoder.summary()

## 8) Pre-train Autoencoder

In [ ]:
# Fungsi untuk menghapus label (autoencoder belajar rekonstruksi gambar)
def strip_labels(ds):
    """Ubah (image, label) menjadi (image, image)"""
    return ds.map(lambda x, y: (x, x))

print("Mulai pre-training Autoencoder")

# Training autoencoder dengan ModelCheckpoint
history_ae = autoencoder.fit(
    strip_labels(train_ds),
    validation_data=strip_labels(val_ds),
    epochs=500,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=4,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            'autoencoder_best.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]
)

print("\nPre-training Autoencoder selesai!")
print("Model terbaik disimpan: autoencoder_best.keras")

In [ ]:
import matplotlib.pyplot as plt

# Plot grafik loss autoencoder
plt.figure(figsize=(12, 4))

# Plot Training & Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history_ae.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history_ae.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Autoencoder - Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot epoch terbaik
best_epoch = history_ae.history['val_loss'].index(min(history_ae.history['val_loss']))
plt.subplot(1, 2, 2)
plt.bar(['Training Loss', 'Validation Loss'], 
        [history_ae.history['loss'][best_epoch], history_ae.history['val_loss'][best_epoch]],
        color=['#3498db', '#e74c3c'])
plt.title(f'Loss pada Epoch Terbaik (Epoch {best_epoch + 1})', fontsize=14, fontweight='bold')
plt.ylabel('Loss (MSE)')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('autoencoder_training.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nGrafik disimpan: autoencoder_training.png")
print(f"Epoch terbaik: {best_epoch + 1}")
print(f"Best Validation Loss: {min(history_ae.history['val_loss']):.6f}")

## 9) Keras Tuner — cari LR terbaik

In [ ]:
import keras_tuner as kt
from tensorflow.keras import layers, Model

def build_model_for_tuning(hp: kt.HyperParameters):
    """
    Fungsi untuk Keras Tuner mencari hyperparameter terbaik
    Hyperparameter yang dicari:
    - Learning rate
    - Dropout rate
    - Jumlah units di dense layer
    """
    # Hyperparameter yang akan dicari
    lr = hp.Float('learning_rate', min_value=1e-5, max_value=5e-3, sampling='log')
    dropout = hp.Choice('dropout', values=[0.2, 0.3, 0.4])
    dense_units = hp.Choice('dense_units', values=[64, 128, 256])
    
    # Gunakan encoder yang sudah di-pretrain (dibekukan)
    enc = build_encoder(IMG_SIZE + (3,), 128)
    enc.set_weights(encoder.get_weights())
    enc.trainable = False  # Freeze encoder
    
    # Bangun model klasifikasi
    inputs = keras.Input(shape=IMG_SIZE + (3,))
    
    # Preprocessing
    x = augment(inputs)
    x = preprocess(x)
    
    # Extract features dengan encoder
    z = enc(x)
    
    # Classifier head
    x = layers.Dropout(dropout)(z)
    x = layers.Dense(dense_units, activation='relu')(x)
    out = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    # Compile model
    model = Model(inputs, out)
    model.compile(
        optimizer=keras.optimizers.Adam(lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Setup Keras Tuner
print("Memulai pencarian hyperparameter terbaik dengan Keras Tuner...")
print("Proses ini akan memakan waktu cukup lama...\n")

tuner = kt.Hyperband(
    build_model_for_tuning,
    objective='val_accuracy',
    max_epochs=12,
    factor=3,
    directory='kt_tuning',
    project_name='lr_search'
)

# Callback untuk early stopping
stop = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=4,
    restore_best_weights=True
)

# Mulai pencarian
tuner.search(train_ds, validation_data=val_ds, callbacks=[stop], verbose=1)

# Ambil hyperparameter terbaik
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\nHyperparameter terbaik:")
print(f"  - Learning Rate: {best_hp.get('learning_rate'):.6f}")
print(f"  - Dropout      : {best_hp.get('dropout')}")
print(f"  - Dense Units  : {best_hp.get('dense_units')}")

## 10) Train Stage-1 (encoder beku)

In [ ]:
# Ambil hyperparameter terbaik dari tuning
best_lr = best_hp.get('learning_rate')
best_dropout = best_hp.get('dropout')
best_dense = best_hp.get('dense_units')

print("Membangun model klasifikasi Stage-1 (encoder dibekukan)...")

# Bangun encoder (dibekukan, tidak dilatih)
enc = build_encoder(IMG_SIZE + (3,), 128)
enc.set_weights(encoder.get_weights())
enc.trainable = False  # Freeze semua layer encoder

# Bangun model klasifikasi lengkap
inputs = keras.Input(shape=IMG_SIZE + (3,))

# Data augmentation dan preprocessing
x = augment(inputs)
x = preprocess(x)

# Extract features dengan encoder
z = enc(x)

# Classifier head
x = layers.Dropout(best_dropout)(z)
x = layers.Dense(best_dense, activation='relu')(x)
out = layers.Dense(NUM_CLASSES, activation='softmax')(x)

# Model lengkap
clf = Model(inputs, out, name='classifier_stage1')

# Compile model
clf.compile(
    optimizer=keras.optimizers.Adam(best_lr),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Mulai training Stage-1...")
print("Training dengan encoder beku...\n")

# Training Stage-1
hist1 = clf.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=6,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            'clf_stage1_best.keras',
            save_best_only=True,
            monitor='val_accuracy',
            verbose=1
        )
    ]
)

print("\nTraining Stage-1 selesai!")

In [ ]:
# Plot grafik training Stage-1
plt.figure(figsize=(14, 5))

# Plot Loss
plt.subplot(1, 3, 1)
plt.plot(hist1.history['loss'], label='Training Loss', linewidth=2, color='#3498db')
plt.plot(hist1.history['val_loss'], label='Validation Loss', linewidth=2, color='#e74c3c')
plt.title('Stage-1: Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot Accuracy
plt.subplot(1, 3, 2)
plt.plot(hist1.history['accuracy'], label='Training Accuracy', linewidth=2, color='#2ecc71')
plt.plot(hist1.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='#f39c12')
plt.title('Stage-1: Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot hasil terbaik
best_epoch_s1 = hist1.history['val_accuracy'].index(max(hist1.history['val_accuracy']))
plt.subplot(1, 3, 3)
metrics = ['Train Acc', 'Val Acc', 'Train Loss', 'Val Loss']
values = [
    hist1.history['accuracy'][best_epoch_s1],
    hist1.history['val_accuracy'][best_epoch_s1],
    hist1.history['loss'][best_epoch_s1],
    hist1.history['val_loss'][best_epoch_s1]
]
colors = ['#2ecc71', '#f39c12', '#3498db', '#e74c3c']
plt.bar(metrics, values, color=colors, alpha=0.8)
plt.title(f'Metrics Terbaik (Epoch {best_epoch_s1 + 1})', fontsize=14, fontweight='bold')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('stage1_training.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nGrafik disimpan: stage1_training.png")
print(f"Epoch terbaik: {best_epoch_s1 + 1}")
print(f"Best Validation Accuracy: {max(hist1.history['val_accuracy']):.4f} ({max(hist1.history['val_accuracy'])*100:.2f}%)")
print(f"Model terbaik disimpan: clf_stage1_best.keras")

## 11) Fine-tune Stage-2 (unfreeze sebagian)

In [ ]:
print("Unfreeze 6 layer terakhir encoder untuk fine-tuning...")

# Unfreeze 6 layer terakhir encoder
for layer in enc.layers[-6:]:
    if hasattr(layer, 'trainable'):
        layer.trainable = True

# Bangun ulang model dengan encoder yang sudah di-unfreeze
inputs2 = keras.Input(shape=IMG_SIZE + (3,))

# Data augmentation dan preprocessing
x = augment(inputs2)
x = preprocess(x)

# Extract features dengan encoder (sebagian trainable)
z = enc(x)

# Classifier head
x = layers.Dropout(best_dropout)(z)
x = layers.Dense(best_dense, activation='relu')(x)
out = layers.Dense(NUM_CLASSES, activation='softmax')(x)

# Model fine-tuned
clf_ft = Model(inputs2, out, name='classifier_stage2')

# Learning rate untuk fine-tuning (lebih kecil)
ft_lr = max(best_lr / 5.0, 1e-5)

# Compile model
clf_ft.compile(
    optimizer=keras.optimizers.Adam(ft_lr),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model siap dengan LR fine-tuning: {ft_lr:.6f}\n")
print("Mulai training Stage-2 (Fine-tuning)...")
print("Training dengan encoder sebagian trainable...\n")

# Training Stage-2
hist2 = clf_ft.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=6,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            'clf_stage2_finetuned.keras',
            save_best_only=True,
            monitor='val_accuracy',
            verbose=1
        )
    ]
)

print("\nFine-tuning Stage-2 selesai!")

In [ ]:
# Plot grafik training Stage-2
plt.figure(figsize=(14, 5))

# Plot Loss
plt.subplot(1, 3, 1)
plt.plot(hist2.history['loss'], label='Training Loss', linewidth=2, color='#3498db')
plt.plot(hist2.history['val_loss'], label='Validation Loss', linewidth=2, color='#e74c3c')
plt.title('Stage-2 (Fine-tuning): Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot Accuracy
plt.subplot(1, 3, 2)
plt.plot(hist2.history['accuracy'], label='Training Accuracy', linewidth=2, color='#2ecc71')
plt.plot(hist2.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='#f39c12')
plt.title('Stage-2 (Fine-tuning): Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot hasil terbaik
best_epoch_s2 = hist2.history['val_accuracy'].index(max(hist2.history['val_accuracy']))
plt.subplot(1, 3, 3)
metrics = ['Train Acc', 'Val Acc', 'Train Loss', 'Val Loss']
values = [
    hist2.history['accuracy'][best_epoch_s2],
    hist2.history['val_accuracy'][best_epoch_s2],
    hist2.history['loss'][best_epoch_s2],
    hist2.history['val_loss'][best_epoch_s2]
]
colors = ['#2ecc71', '#f39c12', '#3498db', '#e74c3c']
plt.bar(metrics, values, color=colors, alpha=0.8)
plt.title(f'Metrics Terbaik (Epoch {best_epoch_s2 + 1})', fontsize=14, fontweight='bold')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('stage2_training.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nGrafik disimpan: stage2_training.png")
print(f"Epoch terbaik: {best_epoch_s2 + 1}")
print(f"Best Validation Accuracy: {max(hist2.history['val_accuracy']):.4f} ({max(hist2.history['val_accuracy'])*100:.2f}%)")
print(f"Model terbaik disimpan: clf_stage2_finetuned.keras")

## 12) Evaluasi & ringkasan

In [ ]:
# Perbandingan Stage-1 vs Stage-2
plt.figure(figsize=(16, 5))

# Perbandingan Loss
plt.subplot(1, 3, 1)
plt.plot(hist1.history['val_loss'], label='Stage-1 (Frozen)', linewidth=2, color='#3498db', marker='o', markersize=4)
plt.plot(hist2.history['val_loss'], label='Stage-2 (Fine-tuned)', linewidth=2, color='#e74c3c', marker='s', markersize=4)
plt.title('Perbandingan Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Perbandingan Accuracy
plt.subplot(1, 3, 2)
plt.plot(hist1.history['val_accuracy'], label='Stage-1 (Frozen)', linewidth=2, color='#2ecc71', marker='o', markersize=4)
plt.plot(hist2.history['val_accuracy'], label='Stage-2 (Fine-tuned)', linewidth=2, color='#f39c12', marker='s', markersize=4)
plt.title('Perbandingan Validation Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Bar chart perbandingan hasil akhir
plt.subplot(1, 3, 3)
categories = ['Val Accuracy', 'Val Loss']
stage1_results = [max(hist1.history['val_accuracy']), min(hist1.history['val_loss'])]
stage2_results = [max(hist2.history['val_accuracy']), min(hist2.history['val_loss'])]

x = range(len(categories))
width = 0.35

plt.bar([i - width/2 for i in x], stage1_results, width, label='Stage-1 (Frozen)', color='#3498db', alpha=0.8)
plt.bar([i + width/2 for i in x], stage2_results, width, label='Stage-2 (Fine-tuned)', color='#e74c3c', alpha=0.8)

plt.title('Hasil Terbaik: Stage-1 vs Stage-2', fontsize=14, fontweight='bold')
plt.ylabel('Value')
plt.xticks(x, categories)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Tambahkan nilai di atas bar
for i, (v1, v2) in enumerate(zip(stage1_results, stage2_results)):
    plt.text(i - width/2, v1, f'{v1:.4f}', ha='center', va='bottom', fontsize=9)
    plt.text(i + width/2, v2, f'{v2:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('stage_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Ringkasan perbandingan
print("\n" + "="*60)
print("📊 RINGKASAN PERBANDINGAN STAGE-1 vs STAGE-2")
print("="*60)
print(f"\n{'Metric':<30} {'Stage-1':<15} {'Stage-2':<15} {'Improvement'}")
print("-"*60)

best_val_acc_s1 = max(hist1.history['val_accuracy'])
best_val_acc_s2 = max(hist2.history['val_accuracy'])
improvement_acc = ((best_val_acc_s2 - best_val_acc_s1) / best_val_acc_s1) * 100

print(f"{'Best Validation Accuracy':<30} {best_val_acc_s1:.4f}        {best_val_acc_s2:.4f}        {improvement_acc:+.2f}%")

best_val_loss_s1 = min(hist1.history['val_loss'])
best_val_loss_s2 = min(hist2.history['val_loss'])
improvement_loss = ((best_val_loss_s2 - best_val_loss_s1) / best_val_loss_s1) * 100

print(f"{'Best Validation Loss':<30} {best_val_loss_s1:.4f}        {best_val_loss_s2:.4f}        {improvement_loss:+.2f}%")
print("-"*60)

if best_val_acc_s2 > best_val_acc_s1:
    print("✓ Fine-tuning meningkatkan performa model!")
else:
    print("⚠ Fine-tuning tidak meningkatkan performa (overfitting?)")
    
print("\n📊 Grafik perbandingan disimpan: stage_comparison.png")

In [ ]:
import json
import datetime

print("\n" + "="*60)
print("📊 EVALUASI MODEL FINAL")
print("="*60)
print("\n⏳ Evaluasi model pada validation set...\n")

# Evaluasi model
res = clf_ft.evaluate(val_ds, return_dict=True)

print("\n✓ Hasil evaluasi model final:")
print(f"  - Loss    : {res['loss']:.4f}")
print(f"  - Accuracy: {res['accuracy']:.4f} ({res['accuracy']*100:.2f}%)")

# Buat ringkasan training
summary = {
    'timestamp': datetime.datetime.now().isoformat(),
    'num_classes': int(len(classes)),
    'class_names': classes,
    'img_size': IMG_SIZE,
    'batch_size': int(32),
    'best_hyperparameters': {
        'learning_rate': float(best_lr),
        'dropout': float(best_dropout) if isinstance(best_dropout, (int, float)) else best_dropout,
        'dense_units': int(best_dense)
    },
    'fine_tuning_lr': float(ft_lr),
    'autoencoder': {
        'best_epoch': int(best_epoch + 1),
        'best_val_loss': float(min(history_ae.history['val_loss']))
    },
    'stage1_frozen': {
        'best_epoch': int(best_epoch_s1 + 1),
        'best_val_accuracy': float(max(hist1.history['val_accuracy'])),
        'best_val_loss': float(min(hist1.history['val_loss']))
    },
    'stage2_finetuned': {
        'best_epoch': int(best_epoch_s2 + 1),
        'best_val_accuracy': float(max(hist2.history['val_accuracy'])),
        'best_val_loss': float(min(hist2.history['val_loss']))
    },
    'final_validation_metrics': res
}

# Simpan ringkasan ke file JSON
with open('training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n✓ Ringkasan training disimpan: training_summary.json")

# Daftar file yang disimpan
print("\n" + "="*60)
print("💾 FILE YANG DISIMPAN:")
print("="*60)
print("\n📁 Model:")
print("  - autoencoder_best.keras")
print("  - clf_stage1_best.keras")
print("  - clf_stage2_finetuned.keras (Model Final)")

print("\n📊 Grafik:")
print("  - autoencoder_training.png")
print("  - stage1_training.png")
print("  - stage2_training.png")
print("  - stage_comparison.png")

print("\n📄 Laporan:")
print("  - training_summary.json")

print("\n" + "="*60)
print("🎉 SEMUA PROSES SELESAI!")
print("="*60)